In [1]:
import os
from tqdm import tqdm
from PIL import Image
import csv

In [2]:
import torch
from torchvision import models
from torchvision import transforms

In [5]:
import numpy as np
from numpy.linalg import norm
import pandas as pd

<h1 style="font-family: 'Times New Roman'">Reshaping and Cleaning</h1>

Before we vectorize the images, we must format them properly for the CNN. Images are reshaped to (3, 224, 224) and saved in the directory inputImagesCNN.


In [6]:
# needed input dimensions for the CNN
inputDim = (224,224)
inputDir = "./images"
inputDirCNN = "inputImagesCNN"

os.makedirs(inputDirCNN, exist_ok = True)

transformationForCNNInput = transforms.Compose([transforms.Resize(inputDim)])

# os.walk to extract the images from the root folder 
paths = {}
for path, dir, imageNames in os.walk(inputDir):
    for imageName in imageNames:
        # ignores invisible files
        if imageName.startswith("."):
            continue
        # Open image and convert any grayscale to RBG.
        I = Image.open(os.path.join(path, imageName)).convert('RGB')
        newI = transformationForCNNInput(I)

        paths[imageName] = path

        # sav images to CNN path
        newI.save(os.path.join(inputDirCNN, imageName))
        
        newI.close()
        I.close()

In [7]:
with open('paths.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    for key, value in paths.items():
       writer.writerow([key, value])

<h1 style="font-family: 'Times New Roman'">Vecorization Algorithm</h1>

Vectorization is the process of turning some input into a vector. I am using the torch Img2VecResnet18 CNN to vectorize the image data. It is a pretrained network The model takes in a (3, 224, 224) image file and returns a (1, 512) array.

In [8]:
from resources import Img2VecResnet18

<h1 style="font-family: 'Times New Roman'">Vectorize Dataset</h1>

Applies our vectorization algorithm to the image dataset and creates a dictionary with the filenames as keys and the vectors as the values.

In [13]:
import numpy
# generate vectors for all the images in the set
img2vec = Img2VecResnet18() 

allVectors = {}
print("Converting images to feature vectors:")
for image in tqdm(os.listdir("inputImagesCNN")):
    if image.startswith("."):
        continue
    I = Image.open(os.path.join("inputImagesCNN", image))
    vec = img2vec.getVec(I)
    allVectors[image] = vec
    I.close() 

Converting images to feature vectors:


100%|██████████| 8446/8446 [04:30<00:00, 31.26it/s]


<h1 style="font-family: 'Times New Roman'">Store Vectors as CSV</h1>


In [15]:
np.savez_compressed('vectors.npz', **allVectors)

<h1 style="font-family: 'Times New Roman'">Shape and Clean Search Images</h1>

Shapes search images into (3, 224, 224) file. 

<h1 style="font-family: 'Times New Roman'">

In [ ]:
inputDim = (224,224)
searchDir = "./searchImages"
searchDirCNN = "searchImagesCNN"

os.makedirs(searchDirCNN, exist_ok = True)

transformationForCNNInput = transforms.Compose([transforms.Resize(inputDim)])

for path, dir, imageNames in os.walk(searchDir):
    for imageName in imageNames:
        # ignores invisible files
        if imageName.startswith("."):
            continue
        # Open image and convert any grayscale to RBG.
        I = Image.open(os.path.join(path, imageName)).convert('RGB')
        newI = transformationForCNNInput(I)

        # sav images to CNN path
        newI.save(os.path.join(searchDirCNN, imageName))
        
        newI.close()
        I.close()

<h1 style="font-family: 'Times New Roman'">Vectorize Search Images</h1>

Applies the CNN vectorization to the folder of search images

In [ ]:
img2vec = Img2VecResnet18() 

searchVectors = {}
print("Converting images to feature vectors:")
for image in tqdm(os.listdir("searchImagesCNN")):
    if image.startswith("."):
        continue
    I = Image.open(os.path.join("searchImagesCNN", image)).convert('RGB')
    vec = img2vec.getVec(I)
    searchVectors[image] = vec
    I.close()

<h1 style="font-family: 'Times New Roman'">Cosine Similarity</h1>

Cosine similarity is a method of measuring the similarity between two vectors. It returns a value from 0 to 1 with 1 meaning most similar and 0 meaning least.

In [ ]:
from resources import cosine_similarity

<h1 style="font-family: 'Times New Roman'">Print Search Options</h1>

Currently, query options are stored in a seperate folder for easy access and use. This cell iterates through this folder printing the names and images of the search options.

In [ ]:
for image in searchVectors.keys():
    print(image + ":")
    im = Image.open(searchDir + "/" + image)
    display(im)

<h1 style="font-family: 'Times New Roman'">Image Query and Search</h1>

In [ ]:
query = "Frida_Kahlo_3.jpg"
v1 = searchVectors[query]
#make a list of images
similarity=list(allVectors.keys())
#sort list by similarity to query
similarity.sort(key = lambda x: cosine_similarity(v1, allVectors[x]), reverse = True)

<h1 style="font-family: 'Times New Roman'">Image Similarities</h1>

Using the vectors generated with our CNN and cosine similarity, we will fins the most similar images in the dataset to our query. The first image is the query used for our search, followed by the most similar photos to the query. If the query is contained in the dataset, the closest image should return itself with a similarity of 1.

In [ ]:
print("Search query:")
img = Image.open(os.path.join(searchDir, query))
display(img)
print("Related Images: ")
#print 10 most similar images
for image in similarity[:10]:
    print(image)
    print("similarity: " + str(cosine_similarity(searchVectors[query], allVectors[image])))
    img = Image.open(os.path.join(paths[image], image))
    display(img)